### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    end = str(today - relativedelta(days=1))
    start = str((today - relativedelta(months=13)).replace(day=1))
else:
    start = str((today - relativedelta(months=12)).replace(day=1))
    end = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))

### QUERIES

In [4]:
q = '''WITH dates_table AS (
    SELECT DISTINCT FORMAT_DATE('%Y-%m',d.date) AS month
    FROM `peya-bi-tools-pro.il_core.dim_date` AS d
    WHERE d.date BETWEEN DATE('{0}') AND DATE('{1}')), 
    cross_table AS (
    SELECT DISTINCT dt.month AS month,
           o.country_id AS country,
           o.user.id AS user
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    CROSS JOIN dates_table AS dt
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND o.order_status= 'CONFIRMED'),
    orders_table AS (
    SELECT FORMAT_DATE('%Y-%m',o.registered_date) AS month,
           o.user.id AS user,
           o.country_id AS country,
           COUNT(DISTINCT o.order_id) AS orders
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2,3),
    users_table AS (
    SELECT ct.month AS month,
           ct.user AS user,
           ct.country AS country,
           IFNULL(ot.orders,0) AS orders
    FROM cross_table AS ct
    LEFT JOIN orders_table AS ot ON ct.user = ot.user AND ct.month = ot.month AND ct.country = ot.country),
    final_table AS (
    SELECT ut.month AS month,
           ut.user AS user,
           ut.country AS country,
           ut.orders AS orders,
           LAG(ut.orders,1,0) OVER (PARTITION BY ut.user,ut.country ORDER BY ut.month ASC) AS orders_lm
    FROM users_table AS ut)
SELECT cn.country_name AS Country,
       ft.month AS Month,
       COUNT(DISTINCT CASE WHEN ft.orders > 0 THEN ft.user ELSE NULL END) AS Active_Users,
       SUM(ft.orders) AS Confirmed_Orders,
       COUNT(DISTINCT CASE WHEN ft.orders >= 7 THEN ft.user ELSE NULL END) AS Users_Over7_TM,
       COUNT(DISTINCT CASE WHEN ft.orders_lm >= 7 THEN ft.user ELSE NULL END) AS Users_Over7_LM,
       SUM(CASE WHEN ft.orders >= 7 THEN ft.orders ELSE 0 END) AS Orders_Over7_TM,
       SUM(CASE WHEN ft.orders_lm >= 7 THEN ft.orders_lm ELSE 0 END) AS Orders_Over7_LM,
       COUNT(DISTINCT CASE WHEN ft.orders >= 7 AND ft.orders_lm >= 7 THEN ft.user ELSE NULL END) AS Repeated_Users_Over7,
       SUM(CASE WHEN ft.orders >= 7 AND ft.orders_lm >= 7 THEN ft.orders ELSE 0 END) AS Repeated_Orders_Over7_TM,
       SUM(CASE WHEN ft.orders >= 7 AND ft.orders_lm >= 7 THEN ft.orders_lm ELSE 0 END) AS Repeated_Orders_Over7_LM
FROM final_table AS ft
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON ft.country = cn.country_id
GROUP BY 1,2
ORDER BY 1,2 ASC'''.format(start,end)

In [5]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q, project_id='peya-argentina', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 539.16rows/s]


In [6]:
# Copio las bases
crudo = bq_crudo.copy()

### TRABAJO

In [7]:
# Creo las columnas faltantes
crudo['Freq'] = crudo['Confirmed_Orders'] / crudo['Active_Users']
crudo['Freq7_TM'] = crudo['Orders_Over7_TM'] / crudo['Users_Over7_TM']
crudo['Freq7_LM'] = crudo['Orders_Over7_LM'] / crudo['Users_Over7_LM']
crudo['Repeating_Users7'] = crudo['Repeated_Users_Over7'] / crudo['Users_Over7_LM']
crudo['Freq_Repeating_TM'] = crudo['Orders_Over7_TM'] / crudo['Repeated_Users_Over7']
crudo['Freq_Repeating_LM'] = crudo['Orders_Over7_LM'] / crudo['Repeated_Users_Over7']

In [8]:
# Reemplazo por 0 los outliers
crudo.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [9]:
# Hago un drop del primer mes (no sirve para calcular)
crudo = crudo[crudo['Month'] != crudo['Month'].min()].copy()

### CARGA

In [10]:
carga(crudo,'1ajuNIPZXJIgT393ySCK7_Q_xWEg5Yjb10TlxJgkwYog','Crudo')

Carga Correcta!
